In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
os.getcwd()

'C:\\Users\\omprakash.bankol\\Desktop\\MARS_Russia\\Analytical Dataset Creation\\Code'

In [2]:
Nielsen_data = pd.read_csv(r"../Data/Raw Data/(BARS)MERGED 2017-2020 NIELSEN BARS DATA LONG FORMAT.csv")
sku_ppg =pd.read_csv(r"../Data/MappingFile/SKU_PPG_Mapping.csv")
week_mapping = pd.read_excel(r"../Data/MappingFile//Week Mapping.xlsx", sheet_name="Mapping")
Nielsen_data = Nielsen_data.loc[Nielsen_data["Manufacturer"]== "MARS"]


In [3]:
Nielsen_data["Nielsen_Sales"] = Nielsen_data["1:Value (in 1000 RUR)"]*1000
Nielsen_data["Nielsen_Units"] = Nielsen_data["1:Units (in 1000 PACKS)"]*1000
Nielsen_data["Nielsen_Volume"] = Nielsen_data["1:Volume (in 1000)"]*1000
Nielsen_data["Nielsen_Numeric_dist"] = Nielsen_data["1:Numeric Distribution (w)"]
Nielsen_data["Nielsen_Weighted_dist"] = Nielsen_data["3:Weighted Distribution (w)"]
#Nielsen_data$Nielsen_Units <- Nielsen_data$`1:Units (in 1000 PACKS)`*1000
#Nielsen_data$Nielsen_Volume  <- Nielsen_data$`1:Volume (in 1000)`*1000
#Nielsen_data$Nielsen_Numeric_dist <- Nielsen_data$`1:Numeric Distribution (w)`
#Nielsen_data$Nielsen_Weighted_dist <-Nielsen_data$`3:Weighted Distribution (w)`


In [4]:
cols = ["Manufacturer","Brand","SKU","Week","Nielsen_Sales","Nielsen_Units","Nielsen_Volume","Nielsen_Numeric_dist","Nielsen_Weighted_dist"]
Nielsen_data = Nielsen_data[cols]

In [5]:
#week_mapping

In [6]:
week_mapping['Week_no'] = week_mapping['Week']

In [7]:
Nielsen_data['Year'] = pd.to_numeric(Nielsen_data['Week'].str.slice(2,6))
Nielsen_data['Week_no'] = pd.to_numeric(Nielsen_data['Week'].str.slice(7,9))
Nielsen_data['Week_no'] = "W"+pd.to_numeric(Nielsen_data['Week_no']).astype(str)

In [8]:
#Nielsen_data <- merge(Nielsen_data,Week_Mapping,by.x=c("Year","Week_no"),by.y = c("Year","Week"),all.x = T)
#Nielsen_data$Date <-as.Date(Nielsen_data$`Week Start date`)

Nielsen_data = pd.merge(Nielsen_data,week_mapping, on=["Year", "Week_no"], how='left')

In [9]:
Nielsen_data['Date'] = pd.to_datetime(Nielsen_data['Week Start date'])

In [10]:
Nielsen_data.shape

#Nielsen_data <-merge(Nielsen_data,sku_ppg,by.x = "SKU",by.y = "Nielsen_SKU",all.x = T)
#Nielsen_data <- Nielsen_data[!is.na(Nielsen_data$PPG),]

(8694, 16)

In [11]:
Nielsen_data = pd.merge(Nielsen_data,sku_ppg, left_on=["SKU"], right_on=["Nielsen_SKU"],how='left')
#pd.merge(df1,df2, left_on=['A','B'],right_on=['A','CC'])

In [12]:
Nielsen_data.shape

(8694, 19)

In [13]:
Nielsen_data = Nielsen_data[Nielsen_data['PPG'].notna()]

In [14]:
sales_sku = pd.read_excel(r"../Data/MappingFile/Choco_NielsenPOS_Hypercom_mapping.xlsx",sheet_name= "Choco_Nielsen_POS_Hypercom",skiprows = 2)
sales_sku.shape

(245, 31)

In [15]:
sales_sku = sales_sku.drop_duplicates()
sales_sku.shape

(245, 31)

In [16]:
x5_weekly = pd.read_excel("../Data/Raw Data/Pyaterochka by week_by sku cases correct.xlsx",sheet_name = "Data")
#setnames(x5_weekly,"...19","Date")
#x5_weekly$Date <-as.Date(x5_weekly$Date)
x5_weekly['Date'] = pd.to_datetime(x5_weekly['Unnamed: 18'])
x5_weekly = x5_weekly.groupby(by = ["Segment","Customer","Category","SKU","Year","Period","Week","SKU Mars","ZREP","Date"]).sum()

In [17]:
x5_weekly = x5_weekly.reset_index()
x5_weekly = x5_weekly.rename(columns={"Items": "X5_Items_Weekly"})
x5_weekly.drop(['Apollo SKU', 'Cases','Unnamed: 12', 'Вес', 'Коэф', 'Unnamed: 15', 'Unnamed: 19'],axis=1,inplace=True)

In [18]:
#sales_sku1 <- sales_sku[!duplicated(sales_sku$POS_X5_SKU.Mars),]
#sales_sku1 <-sales_sku1[!is.na(sales_sku1$POS_X5_SKU.Mars),]
sales_sku1 = sales_sku.drop_duplicates(subset='POS X5_SKU.Mars')
sales_sku1 = sales_sku1[sales_sku1['POS X5_SKU.Mars'].notna()]

In [19]:
#Get Nielsen SKU map
#x5_weekly1 <-merge(x5_weekly,sales_sku1[,c("Nielsen_SKU","POS_X5_SKU.Mars")],by.x="SKU Mars",by.y = "POS_X5_SKU.Mars",all.x = T)
x5_weekly1 = pd.merge(x5_weekly,sales_sku1[["Nielsen_SKU","POS X5_SKU.Mars"]], left_on=["SKU Mars"], right_on=["POS X5_SKU.Mars"],how='left')

In [20]:
#Nielsen_data1 <- merge(Nielsen_data,x5_weekly1[,c("X5_Items_Weekly","Nielsen_SKU","Date","SKU Mars")],by.x = c("SKU","Date"),by.y = c("Nielsen_SKU","Date"),all.x = T)
Nielsen_data1 = pd.merge(Nielsen_data,x5_weekly1[["X5_Items_Weekly","Nielsen_SKU","Date"]],left_on=["SKU","Date"], right_on=["Nielsen_SKU","Date"],how='left')

In [21]:
#setnames(Nielsen_data1,c("SKU Mars"),c("SKU_X5_Weekly"))
Nielsen_data1 = Nielsen_data1.rename(columns={"SKU Mars": "SKU_X5_Weekly"})

In [22]:
x5_monthly = pd.read_csv(r"../Data/Raw Data/5 Ka Sales Out_Monthly.csv",low_memory=False)


In [23]:
#x5_monthly <- x5_monthly %>% group_by(Banner,Segment,`Brand Group`,`Product name`,
#                                      `National name`,Year_Month) %>% summarise(X5_Items_Monthly=sum(`Sum of Items`,na.rm = T),
#                                                                                X5_LSV_Monthly=sum(`LSV (rub)`,na.rm = T))
cls = ["Banner","Segment","Brand Group","Product name","National name","Year_Month"]
for i in cls:
    x5_monthly[i] = x5_monthly[i].astype(str)
x5_monthly = x5_monthly.groupby(by=["Banner","Segment","Brand Group","Product name","National name","Year_Month"]).sum()
x5_monthly = x5_monthly.reset_index()
x5_monthly = x5_monthly[['Banner', 'Segment', 'Brand Group', 'Product name', 'National name','Year_Month','LSV (rub)','Sum of Items']]
x5_monthly = x5_monthly.rename(columns={"LSV (rub)": "X5_LSV_Monthly"})
x5_monthly = x5_monthly.rename(columns={"Sum of Items": "X5_Items_Monthly"})

In [24]:
x5_monthly = x5_monthly.loc[(x5_monthly["Banner"]=="5ka") & (x5_monthly["Segment"]=="CHOCO")]

In [25]:
#sales_sku2 <- sales_sku[!duplicated(sales_sku$POS.X5.MONTHLY_National.name),]
#sales_sku2 <-sales_sku2[!is.na(sales_sku2$`POS.X5.MONTHLY_National.name`),]
sales_sku2 = sales_sku.drop_duplicates(subset='POS X5 Monthly_National name')
sales_sku2 = sales_sku2[sales_sku2['POS X5 Monthly_National name'].notna()]

In [26]:
#Get Nielsen SKU map
#x5_monthly1 <-merge(x5_monthly,sales_sku2[,c("Nielsen_SKU","POS.X5.MONTHLY_National.name")],by.x="National name",by.y="POS.X5.MONTHLY_National.name",all.x=T)
x5_monthly1 = pd.merge(x5_monthly,sales_sku2[["Nielsen_SKU","POS X5 Monthly_National name"]], left_on="National name", right_on="POS X5 Monthly_National name", how='left')

In [27]:
#blank due to pivot creation
#x5_monthly1 <-x5_monthly1[x5_monthly1$Year_Month !="",]
#x5_monthly1$Year= as.numeric(lapply(x5_monthly1$Year_Month, function(i) unlist(strsplit(i, split = '[_]'))[1]))
#x5_monthly1$Month= as.numeric(lapply(x5_monthly1$Year_Month, function(i) unlist(strsplit(i, split = '[_]'))[2]))
x5_monthly1 = x5_monthly1.loc[x5_monthly1['Year_Month']!="nan"]
x5_monthly1['Year'] = pd.to_numeric(x5_monthly1['Year_Month'].str.split('_').str[0])
x5_monthly1['Month'] = pd.to_numeric(x5_monthly1['Year_Month'].str.split('_').str[1])

In [28]:
#merge Nielsen and X5 monthly data
#Nielsen_data2 <- merge(Nielsen_data1,x5_monthly1[,c("National name","X5_Items_Monthly","X5_LSV_Monthly","Nielsen_SKU","Month","Year")],by.x = c("SKU","Month","Year"),by.y = c("Nielsen_SKU","Month","Year"),all.x = T)
#setnames(Nielsen_data2,c("National name"),c("SKU_X5_Monthly"))
Nielsen_data2 = pd.merge(Nielsen_data1,x5_monthly1[["National name","X5_Items_Monthly","X5_LSV_Monthly","Nielsen_SKU","Month","Year"]],left_on= ["SKU","Month","Year"],right_on=["Nielsen_SKU","Month","Year"],how='left')
Nielsen_data2 = Nielsen_data2.rename(columns={"National name": "SKU_X5_Monthly"})

In [29]:
magnit_data = pd.read_excel("../Data/Raw Data/Tander Choco - Sales out 2018-2020 (with month)_Data.xlsx")

In [30]:
magnit_data = magnit_data.loc[magnit_data.Format=='МД']
#magnit_data = magnit_data.rename(columns={"Продажи, LSV (rub)": "LSV (rub)"})

In [31]:
#magnit_data <-magnit_data %>% group_by(Format,ProductGroup,`National name`,Month_year) %>% summarise(Magnit_Units_Monthly=sum(`Quantity (items)`,na.rm = T),
#                                                                                                     Magnit_Sales_Monthly=sum(`Sales Value (rub)`,na.rm = T),
#                                                                                                     Magnit_LSV_Monthly=sum(`LSV (rub)`,na.rm = T))
magnit_data = magnit_data.groupby(by=["Format","Brand","Product Name","Year","Month"]).sum()
magnit_data = magnit_data.reset_index()
magnit_data = magnit_data.rename(columns={"Quantity (items)": "Magnit_Units_Monthly"})
magnit_data = magnit_data.rename(columns={"Sales Value (rub)": "Magnit_Sales_Monthly"})
magnit_data = magnit_data.rename(columns={"LSV (rub)": "Magnit_LSV_Monthly"})                                                                                                                            

In [32]:
sales_sku3 = sales_sku.drop_duplicates(subset='POS Tander_Product.Name')
sales_sku3 = sales_sku3[sales_sku3['POS Tander_Product.Name'].notna()]


In [33]:
magnit_data1 = pd.merge(magnit_data,sales_sku3[["Nielsen_SKU","POS Tander_Product.Name"]],left_on="Product Name",right_on="POS Tander_Product.Name",how='left')
#magnit_data1['Year'] = pd.to_numeric(magnit_data1['Month_year'].str.split('_').str[1])
#magnit_data1['Month'] = pd.to_numeric(magnit_data1['Month_year'].str.split('_').str[0])

In [34]:
#merge Nielsen and X5 monthly data
#Nielsen_data3 <- merge(Nielsen_data2,magnit_data1[,c("National name","Magnit_Units_Monthly","Magnit_Sales_Monthly","Magnit_LSV_Monthly","Nielsen_SKU","Month","Year")],by.x = c("SKU","Month","Year"),by.y = c("Nielsen_SKU","Month","Year"),all.x = T)
#setnames(Nielsen_data3,c("National name"),c("SKU_Magnit_Monthly"))
Nielsen_data3 = pd.merge(Nielsen_data2,magnit_data1[["Magnit_Units_Monthly","Magnit_Sales_Monthly","Magnit_LSV_Monthly","Nielsen_SKU","Month","Year"]],left_on= ["SKU","Month","Year"],right_on= ["Nielsen_SKU","Month","Year"],how='left')
Nielsen_data3 = Nielsen_data3.rename(columns={"National name": "SKU_Magnit_Monthly"})

In [35]:
Nielsen_data3.dtypes
Nielsen_data3["X5_Items_Weekly"] = pd.to_numeric(Nielsen_data3["X5_Items_Weekly"])
#Nielsen_data3["X5_LSV_Monthly"] = pd.to_numeric(Nielsen_data3["X5_LSV_Monthly"])
#Nielsen_data3["X5_Items_Monthly"] = pd.to_numeric(Nielsen_data3["X5_Items_Monthly"])


In [36]:
Nielsen_data4 = Nielsen_data3.groupby(by=["Manufacturer","PPG","PPG_Part","Date","Year","Month","Week_no"]).aggregate({"Nielsen_Units":np.sum,"Nielsen_Sales":np.sum,"Nielsen_Volume":np.sum,"Nielsen_Numeric_dist":max,"Nielsen_Weighted_dist":max,"X5_Items_Weekly":np.sum,"X5_Items_Monthly":np.sum,"X5_LSV_Monthly":np.sum,"Magnit_Units_Monthly":np.sum,"Magnit_Sales_Monthly":np.sum,"Magnit_LSV_Monthly":np.sum})

In [37]:
Nielsen_data4 = Nielsen_data4.reset_index()

In [38]:
cls = ['Nielsen_Units','Nielsen_Sales', 'Nielsen_Volume','Nielsen_Numeric_dist', 'Nielsen_Weighted_dist', 'X5_Items_Weekly','X5_Items_Monthly', 'X5_LSV_Monthly', 'Magnit_Units_Monthly','Magnit_Sales_Monthly', 'Magnit_LSV_Monthly']
Nielsen_data4[cls] = Nielsen_data4[cls].replace([0], np.nan)

In [39]:
Nielsen_data4 = Nielsen_data4.sort_values(["PPG","Date"])
Nielsen_data4 = Nielsen_data4.rename(columns={"X5_Items_Weekly": "X5_Units_Weekly"})
Nielsen_data4 = Nielsen_data4.rename(columns={"X5_Items_Monthly": "X5_Units_Monthly"})


In [40]:
Nielsen_data4['Date'] = pd.to_datetime(Nielsen_data4['Date'])
Nielsen_data4['Date'] = Nielsen_data4['Date'].dt.date

In [41]:
Nielsen_data4.to_excel(r"../Process/Analytical_Input_Optimizer/Nielsen_POS_Data_for_Apportion_CHOCO.xlsx",index=False)